In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras import Sequential
from keras import layers
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import adam_v2
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import random
from tqdm.notebook import tqdm
import collections

In [4]:
train_data = np.array(pd.read_pickle('D:/F/df/NoDef/X_train_NoDef.pkl')) # shape (76000, 5000), 95 website, 800 repeat
train_index = np.array(pd.read_pickle('D:/F/df/NoDef/y_train_NoDef.pkl')) # shape (76000,), 0-94 range, random

test_data = np.array(pd.read_pickle('D:/F/df/NoDef/X_test_NoDef.pkl')) # shape (9500, 5000), 95 website, 100 repeat
test_index = np.array(pd.read_pickle('D:/F/df/NoDef/y_test_NoDef.pkl')) # shape (9500,), 0-94 range, sequence

In [5]:
train_data = train_data[0 : 7600]
train_index = train_index[0 : 7600]

In [6]:
def get_data_length(data_vector):
    data_length = 5000
    for i in range(5000):
        if data_vector[i] == 0:
            data_length = i
            break
    return data_length

def add_noise(data_vector, data_length, count):
    for i in range(count):
        location = random.randint(0, data_length - 1)
        packet = random.randint(0, 1) * 2 - 1
        data_vector = np.insert(data_vector, location, packet)
        if data_length < 5000:
            data_length += 1
    
    return data_vector[0 : 5000]

In [7]:
train_data_aug = train_data.copy()
train_index_aug = train_index.copy()
for j in range(1):
    train_data_aug_one = train_data.copy()
    for i in tqdm(range(7600)):
        length = get_data_length(train_data_aug_one[i])
        rate = random.randint(0, 100) / 100
        train_data_aug_one[i] = add_noise(train_data_aug_one[i], length, int(length * rate))
    train_data_aug = np.append(train_data_aug, train_data_aug_one, axis=0)
    train_index_aug = np.append(train_index_aug, train_index, axis=0)

  0%|          | 0/7600 [00:00<?, ?it/s]

In [8]:
# DF model used for non-defended dataset
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.activation import ELU
from keras.initializers import glorot_uniform

class DFNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        #Block1
        filter_num = ['None',32,64,128,256]
        kernel_size = ['None',8,8,8,8]
        conv_stride_size = ['None',1,1,1,1]
        pool_stride_size = ['None',4,4,4,4]
        pool_size = ['None',8,8,8,8]

        model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1], input_shape=input_shape,
                         strides=conv_stride_size[1], padding='same',
                         name='block1_conv1'))
        model.add(BatchNormalization(axis=-1))
        model.add(ELU(alpha=1.0, name='block1_adv_act1'))
        model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1],
                         strides=conv_stride_size[1], padding='same',
                         name='block1_conv2'))
        model.add(BatchNormalization(axis=-1))
        model.add(ELU(alpha=1.0, name='block1_adv_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[1], strides=pool_stride_size[1],
                               padding='same', name='block1_pool'))
        model.add(Dropout(0.1, name='block1_dropout'))

        model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                         strides=conv_stride_size[2], padding='same',
                         name='block2_conv1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block2_act1'))

        model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                         strides=conv_stride_size[2], padding='same',
                         name='block2_conv2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block2_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[2], strides=pool_stride_size[3],
                               padding='same', name='block2_pool'))
        model.add(Dropout(0.1, name='block2_dropout'))

        model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                         strides=conv_stride_size[3], padding='same',
                         name='block3_conv1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block3_act1'))
        model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                         strides=conv_stride_size[3], padding='same',
                         name='block3_conv2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block3_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[3], strides=pool_stride_size[3],
                               padding='same', name='block3_pool'))
        model.add(Dropout(0.1, name='block3_dropout'))

        model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                         strides=conv_stride_size[4], padding='same',
                         name='block4_conv1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block4_act1'))
        model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                         strides=conv_stride_size[4], padding='same',
                         name='block4_conv2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block4_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[4], strides=pool_stride_size[4],
                               padding='same', name='block4_pool'))
        model.add(Dropout(0.1, name='block4_dropout'))

        model.add(Flatten(name='flatten'))
        model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='fc1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='fc1_act'))

        model.add(Dropout(0.7, name='fc1_dropout'))

        model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='fc2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='fc2_act'))

        model.add(Dropout(0.5, name='fc2_dropout'))

        model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc3'))
        model.add(Activation('softmax', name="softmax"))
        return model


In [9]:
model = DFNet.build(input_shape=(5000,1), classes=95)

In [10]:
from keras.optimizers import Adamax

model.compile(loss="categorical_crossentropy", 
              optimizer=Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
              metrics=["accuracy"])

In [11]:
model.fit(train_data_aug.astype('float32'), to_categorical(train_index_aug), epochs = 30, validation_split=0.3)

Epoch 1/30
333/333 [==============================] - 119s 353ms/step - loss: 4.0549 - accuracy: 0.0749 - val_loss: 5.3071 - val_accuracy: 0.0105
Epoch 2/30
333/333 [==============================] - 116s 349ms/step - loss: 3.0611 - accuracy: 0.2049 - val_loss: 2.6980 - val_accuracy: 0.2711
Epoch 3/30
333/333 [==============================] - 114s 341ms/step - loss: 2.4209 - accuracy: 0.3510 - val_loss: 2.4219 - val_accuracy: 0.3414
Epoch 4/30
333/333 [==============================] - 114s 342ms/step - loss: 2.0226 - accuracy: 0.4441 - val_loss: 2.1138 - val_accuracy: 0.4079
Epoch 5/30
333/333 [==============================] - 114s 342ms/step - loss: 1.7180 - accuracy: 0.5190 - val_loss: 2.2924 - val_accuracy: 0.3561
Epoch 6/30
333/333 [==============================] - 114s 341ms/step - loss: 1.4925 - accuracy: 0.5825 - val_loss: 1.8100 - val_accuracy: 0.4779
Epoch 7/30
333/333 [==============================] - 114s 341ms/step - loss: 1.3362 - accuracy: 0.6286 - val_loss: 1.4629 -

In [12]:
model.evaluate(test_data, to_categorical(test_index))

297/297 [==============================] - 16s 54ms/step - loss: 0.2567 - accuracy: 0.9419


[0.2566867172718048, 0.9418947100639343]

In [13]:
model.save('data/model_tor_random_aug_repeat.h5')